**# House Price Prediction**

## Data Preprocessing and feature engineering

**Author** - `Prasad Parjane`

**Source**-(https://www.kaggle.com/competitions/house-prices-advanced-regression-techniques/data?select=train.csv)

**Objective** - To predict the sales price of each house using the feature engineering and advanced regression techniques like random forest and gradient boosting



In [ ]:
# import required python libraries